# Single Class Pneumonia

In [1]:
import tensorflow as tf


gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3999)])
 

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
print(('Is your GPU available for use?\n{0}').format(
    'Yes, your GPU is available: True' if tf.test.is_gpu_available() == True else 'No, your GPU is NOT available: False'
))

print(('\nYour devices that are available:\n{0}').format(
    [device.name for device in tf.config.experimental.list_physical_devices()]
))

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Is your GPU available for use?
Yes, your GPU is available: True

Your devices that are available:
['/physical_device:CPU:0', '/physical_device:XLA_CPU:0', '/physical_device:GPU:0', '/physical_device:XLA_GPU:0']


Load and prepare data

In [3]:
import cv2 as cv
import pickle
import random
import numpy as np
 

In [4]:
raw_data = pickle.load( open( "C:/Users/uprz2/Downloads/cs330/data/pneumonia-256-grayscale", "rb" ) ) # array of tuples( [x,y], 0|1 )
random.shuffle(raw_data)


fraction_train = 0.8
train_size = int(len(raw_data) * fraction_train)

x_train = []
y_train = []

for v in raw_data[0: train_size] :
    x_train.append(v[0])
    y_train.append(v[1])

x_train = np.array(x_train) 
y_train = np.array(y_train)    

x_test = []
y_test = []


for v in raw_data[train_size:]:
    x_test.append(v[0])
    y_test.append(v[1])
    

x_test = np.array(x_test) 
y_test = np.array(y_test)  


x_train, x_test = x_train / 255.0, x_test / 255.0



# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

print("train size: {}, test size: {}".format(len(x_train), len(x_test)))


train size: 4684, test size: 1172


Use `tf.data` to batch and shuffle the dataset:

In [5]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(100).batch(16)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(16)

In [6]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(64, activation='relu')
    self.d2 = Dense(2)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

Choose an optimizer and loss function for training: 

In [7]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

Select metrics to measure the loss and the accuracy of the model. These metrics accumulate the values over epochs and then print the overall result.

In [8]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

Use `tf.GradientTape` to train the model:

In [9]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

Test the model:

In [10]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [11]:
EPOCHS = 30

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    'Epoch {}, '.format(epoch + 1),
    'Loss: {}, '.format(train_loss.result()),
    'Accuracy: {}, '.format(train_accuracy.result() * 100),
    'Test Loss: {}, '.format(test_loss.result()),
    'Test Accuracy: {}'.format(test_accuracy.result() * 100)
  )

Epoch 1,  Loss: 1.7011065483093262,  Accuracy: 90.28607940673828,  Test Loss: 0.16085077822208405,  Test Accuracy: 94.02730560302734
Epoch 2,  Loss: 0.09782040864229202,  Accuracy: 96.58411407470703,  Test Loss: 0.17078515887260437,  Test Accuracy: 94.62457275390625
Epoch 3,  Loss: 0.038599178194999695,  Accuracy: 98.84713745117188,  Test Loss: 0.172670379281044,  Test Accuracy: 94.62457275390625
Epoch 4,  Loss: 0.019333399832248688,  Accuracy: 99.48761749267578,  Test Loss: 0.19332608580589294,  Test Accuracy: 94.11262512207031
Epoch 5,  Loss: 0.015553709119558334,  Accuracy: 99.6370620727539,  Test Loss: 0.24246974289417267,  Test Accuracy: 93.77133178710938
Epoch 6,  Loss: 0.007386047393083572,  Accuracy: 99.87190246582031,  Test Loss: 0.19891250133514404,  Test Accuracy: 94.88054656982422
Epoch 7,  Loss: 0.00484649371355772,  Accuracy: 99.91460418701172,  Test Loss: 0.29983845353126526,  Test Accuracy: 92.7474365234375
Epoch 8,  Loss: 0.027490580454468727,  Accuracy: 99.03928375244